In [ ]:
# Importando bibliotecas necessárias

import pandas as pd
import os
import glob
import numpy as np
from datetime import datetime

In [ ]:
# Definindo informações importantes

loja = 1
periodo = 2

In [ ]:
# Leitura dos arquivos da ficha 3

ficha_3_1 = pd.read_excel(f'C:\Bots\Bot_transmissao\Geracao CAT\Loja_{loja}\\ficha_3_loja_{loja}_p{periodo}_parte1_v1.xlsx') 
ficha_3_2 = pd.read_excel(f'C:\Bots\Bot_transmissao\Geracao CAT\Loja_{loja}\\ficha_3_loja_{loja}_p{periodo}_parte2_v1.xlsx')

ficha_3 = pd.concat([ficha_3_1, ficha_3_2])
ficha_3

In [ ]:
def recalcular_digito_verificador(nfe_completa):
    # Verifica se a NF-e com o dígito verificador tem o tamanho correto
    if pd.isnull(nfe_completa) or len(nfe_completa) != 44:
        raise ValueError(f"Valor inválido na coluna 'CHV_DOC': {nfe_completa}")

    # Remove o último caractere (dígito verificador)
    nfe_sem_dv = nfe_completa[:-1]

    # Restante da lógica de cálculo do dígito verificador
    soma = 0
    peso = 2
    nfe_invertida = nfe_sem_dv[::-1]

    for digito in nfe_invertida:
        soma += int(digito) * peso
        peso += 1
        if peso > 9:
            peso = 2

    resto = soma % 11
    dv = 0 if resto == 0 or resto == 1 else 11 - resto

    return dv

df_0000 = pd.read_excel(f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Referencias\\0000_0150_P{periodo}.xlsx', sheet_name='0000', dtype=str)

df_0000['DATA'] = df_0000['DATA'].astype(str).str.slice(0, 8)
df_0000['DATA'] = [datetime.strptime(x, '%Y%m%d').strftime('%m%Y') for x in df_0000['DATA']]
df_0000['COD_VER'] = ['01'] * df_0000.shape[0]
df_0000['COD_FIN'] = ['00'] * df_0000.shape[0]
df_0000 = df_0000[['LOJA','REG', 'DATA', 'NOME', 'CNPJ', 'IE',
        'COD_MUN', 'COD_VER', 'COD_FIN']]

df_0000 = df_0000[df_0000['LOJA'] == f'Loja {loja}']


cnpj_loja = df_0000[df_0000['LOJA'] == f'Loja {loja}']['CNPJ'].values[0]

# ficha_3['CHV_DOC'] = np.where(ficha_3['FONTE'].isin(['entrada_oprprp', 'saida_nfe', 'dfe']),
#                            ficha_3['CHV_DOC'].str.slice(0,6) + cnpj_loja + ficha_3['CHV_DOC'].str.slice(20,44),
#                            ficha_3['CHV_DOC'])
    
# ficha_3['DV_NFe'] = ficha_3['CHV_DOC'].apply(recalcular_digito_verificador)
# ficha_3['CHV_DOC'] = ficha_3['CHV_DOC'].str.slice(0,-1) + ficha_3['DV_NFe'].astype(str)
# ficha_3

In [ ]:
# Lista todos os arquivos TXT na pasta
pasta_arquivos = f'C:\Bots\Bot_transmissao\Geracao CAT\Loja_{loja}\Teste_loja_{loja}'

# Filtra apenas os arquivos que começam com '1100'
arquivos_1100 = [arquivo for arquivo in os.listdir(pasta_arquivos) if arquivo.startswith('1100')]
arquivos_1100
# Lista para armazenar os dataframes de cada arquivo
dataframes = []

# Lê cada arquivo e adiciona ao dataframe
for arquivo in arquivos_1100:
    df = pd.read_csv(os.path.join(pasta_arquivos, arquivo), sep='|', header=None, 
                     names=['Cod_reg', 'Chv_doc', 'data', 'num_item', 'ind_oper', 
                             'cod_prod', 'cfop', 'qtd', 'icms_tot', 'vlr_confr', 'cod_legal'],
                    dtype=str)  # Se o arquivo estiver em formato CSV, ajuste os parâmetros conforme necessário
    dataframes.append(df)

# Concatena todos os dataframes em um único dataframe
df_final = pd.concat(dataframes, ignore_index=True)
df_final

In [ ]:
df_1100_icms_tot = df_final[['Chv_doc', 'num_item', 'icms_tot']]
ficha_3_icms_tot = ficha_3[['CHV_DOC', 'NUM_ITEM', 'ICMS_TOT_PCAT']]

ficha_3_icms_tot['NUM_ITEM'] = ficha_3_icms_tot['NUM_ITEM'].astype(str)
df_1100_icms_tot['num_item'] = df_1100_icms_tot['num_item'].astype(str)

df_1100_icms_tot['Chv_item'] = df_1100_icms_tot['Chv_doc'] + '-' + df_1100_icms_tot['num_item']
ficha_3_icms_tot['CHV_ITEM'] = ficha_3_icms_tot['CHV_DOC'] + '-' + ficha_3_icms_tot['NUM_ITEM']

In [ ]:
df_1100_icms_tot['icms_tot'] = df_1100_icms_tot['icms_tot'].str.replace(',', '.').astype(float)
df_1100_icms_tot

In [ ]:
merged = df_1100_icms_tot.merge(ficha_3_icms_tot[['CHV_ITEM', 'ICMS_TOT_PCAT']],
                               left_on=['Chv_item'],
                               right_on=['CHV_ITEM'])

merged = merged.drop_duplicates()
merged

In [ ]:
merged['Diffs'] = np.abs(merged['icms_tot'] - merged['ICMS_TOT_PCAT'])
merged

In [ ]:
if merged[merged['Diffs'] >= 0.1].shape[0] > 0:
    print('Há erro na geração do TXT em relação ao ICMS_TOT_PCAT, favor verificar')
    sys.exit()

In [ ]:
df_final

In [ ]:
# Checagem de existência de ICMS_TOT

if df_final[(df_final['cfop'].isin(['1403', '1409', '1411', '5411'])) & (df_final['icms_tot'].isnull())].shape[0] > 0:
    print('Existem entradas, devoluções de entrada e devoluções de saída com ICMS_TOT nulo. Favor checar')
    sys.exit()
    
if df_final[(df_final['cfop'].isin(['5409', '5405', '5403', '5401'])) & (df_final['icms_tot'].notnull())].shape[0] > 0:
    print('Existem saídas com campo ICMS_TOT preenchido. Favor checar')
    sys.exit()

In [ ]:
# Checagem de Valor de confronto

if df_final[(df_final['cfop'].isin(['1403', '1409', '5411'])) & (df_final['vlr_confr'].notnull())].shape[0] > 0:
    print('Existem entradas e devoluções de entrada com Valor de Confronto não nulo. Favor, checar.')
    sys.exit()
    
if df_final[(df_final['cfop'].isin(['5405', '1411', '5409', '5403', '5401'])) & (df_final['cod_legal'] == '1') & (df_final['vlr_confr'].isnull())].shape[0] > 0:
    print('Existem sáidas e devoluções de saída com Valor de Confronto nulo. Favor, checar.')
    sys.exit()
    
if df_final[(df_final['cfop'].isin(['5405', '1411', '5409', '5403', '5401'])) & (df_final['cod_legal'] == '0') & (df_final['vlr_confr'].notnull())].shape[0] > 0:
    print('Existem saídas e devoluções de saída com Valor de Confronto não nulo quando Código Legal igual a 0. Favor, checar')
    sys.exit()

In [ ]:
# Cheagem de código legal

if df_final[(df_final['cfop'].isin(['1403', '1409', '5411'])) & (df_final['cod_legal'].notnull())].shape[0] > 0:
    print('Existem entradas e devoluções de entrada com Código Legal não nulo. Favor, checar.')
    sys.exit()
    
if df_final[(df_final['cfop'].isin(['5405', '1411', '5409', '5403', '5401'])) & (df_final['cod_legal'].isnull())].shape[0] > 0:
    print('Existem sáidas e devoluções de saída com Código Legal nulo. Favor, checar.')
    sys.exit()

In [ ]:
df_final[(df_final['cfop'] == '1411') & (df_final['cod_legal'] == '1')]['vlr_confr'].unique()